# Einleitung

Willkommen zu unseren Code-Beispielen für die API! Diese Beispiele zeigen, wie man verschiedene API-Endpunkte nutzt, um Daten abzurufen, Anrufe zu erstellen und Zeitfenster zu reservieren.

Die aktuellste API-Dokumentation finden Sie unter: [ak API Dokumentation - Arztkonsultation ](https://arztkonsultation.de/ak-api-dokumentation/)


**Bitte beachten Sie, dass Sie die API-URL ggf. anpassen müssen.**

## Wie benutze ich das Jupyter Notebook?

Sie können sich die ipynb Datei kopieren oder direkt im Google Colab ausführen.

Sie können dann entweder

1. den Code so anpassen, dass Sie im ersten auskommentierten Abschnitt die E-Mail-Adresse und das Passwort des Users hinterlegen, der die API-Calls ausführen soll. Sie können dann den Code im zweiten Abschnitt "Authentifizierung via config.ini" ignorieren

oder

2. Sie erstellen eine config.ini Datei, hinterlegen die Zugangsdaten des Users und legen die Datei in Google Drive ab. GGf müssen Sie den Pfad im Code ändern



# Authentifizierung und Token Abruf

Zuerst muss sich ein User via E-Mail-Adresse und Password authentifizieren (z.B. der Admin-User), um einen Token für nachfolgende API-Aufrufe zu erhalten.

Das nachfolgende Beispiel sendet eine POST-Anfrage zur Authentifizierung und gibt den Zugriffstoken zurück.



In [ ]:
#Authentifizierung via E-Mail und Passwort

#email='E-Mail-Adresse des User der den API-Call durchführen soll (z.B. Arzt oder Admin)'
#password='Passwort des Accounts'

In [1]:
#Authentifizierung via config.ini
import os
from configparser import ConfigParser
from google.colab import drive
drive.mount('/content/drive')

# Der Pfad zur config.ini Datei
config_file_path = '/content/drive/MyDrive/Colab Notebooks/config.ini'

# Sicherstellen, dass die Datei existiert
if not os.path.exists(config_file_path):
    raise FileNotFoundError(f"Die Datei {config_file_path} wurde nicht gefunden")

# ConfigParser initialisieren und die Datei lesen
config = ConfigParser()
config.read(config_file_path)

# Zugangsdaten auslesen
email = config['credentials']['email']
password = config['credentials']['password']

Mounted at /content/drive


In [2]:
# @title
import requests

#Hostname anpassen!
hostname='app-sandbox.arztkonsultation.de'

def fetch(method, path, data=None, token=None):
    url = f'https://{hostname}{path}'
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    if token:
        headers['Authorization'] = f'Bearer {token}'

    response = requests.request(method, url, json=data, headers=headers)

    if response.status_code >= 400:
        raise Exception(f'Request failed with status code {response.status_code}: {response.text}')

    return response.json()


In [3]:
def login():
    c = {'email': email, 'password': password}
    result = fetch('POST', '/api/v2/auth', c)
    return result['access_token']

token = login()


# User anlegen

Das Skript erstellt einen Benutzer-Account.

Es sendet eine Anfrage an die API, um einen neuen Benutzer mit festgelegten Daten wie Name, E-Mail, Passwort usw. zu erstellen.

In [ ]:
def create_user(token, name, email, mobile, pw):
    companies = fetch('GET', '/api/v2/companies', token=token)
    company_id = companies['data'][0]['id']
    user_data = {
        "name": name,
        "email": email,
        "password": pw,
        "gender": "male",
        "company_id": company_id,
        "langs": [0],
        "roles": [2],
        "mobile": mobile,
        "newsletter": "false"
    }
    user = fetch('POST', '/api/v2/users', user_data, token=token)
    return user['data']


In [ ]:
user=create_user(token, "Testuser2", "Testuser2@arztkonsultation.de", "+49123465789","supergeheim")
user

{'id': 58928,
 'name': 'Testuser2',
 'firstname': '',
 'lastname': 'Testuser2',
 'title': '',
 'created_at': 1718035142,
 'updated_at': 1718035142,
 'deleted_at': None,
 'status': {'no_approbation_reminder_until': None, 'pending_email': None},
 'gender': 'male',
 'company_id': 827,
 'company_name': 'ak Code Beispiele',
 'ousubsidiary_id': None,
 'ousubsidiary_name': '',
 'oulocation_id': None,
 'oulocation_name': '',
 'department_id': None,
 'department_name': '',
 'languages': [],
 'roles': [{'id': 2, 'name': 'doc'}],
 'permissions': [],
 'tan': '',
 'tan_valid_from': '',
 'tan_valid_until': None,
 'tan_by_id': None,
 'tan_by_type': None,
 'tan_created_type': None,
 'consentstatus': {'tan_descriptions_agreed': False, 'tan_data_agreed': False},
 'tags': [],
 'email_verified_at': 1718035142,
 'is_unlocked': 1,
 'expires_at': None,
 'is_app_user': False,
 'description': None,
 'extid': None,
 'newsletter': {'generals': False, 'updates': False},
 'email': 'testuser2@arztkonsultation.de',


# Videocall erstellen


## Videocall zwischen *einem* ak-User & *einem* TAN-User (z.B. Arzt/Ärztin zu Patient:in)

Dieser Code zeigt, wie man einen Vidoecall zwischen einem ak-User (z.B. Arzt/Ärztin) und einem TAN-User (z.B. Patient:in) erstellt.

Je nach Rolle, kann der Account, der den Call erzeugt hat auch Teilnehmer:in sein (z.B. wenn der Account die Rolle "Arzt" hat)

In [ ]:
import requests
import json
from datetime import datetime


def create_call(token):
    call_data = {
        "duration": 15,
        "tan_username": "TAN-User1",
        "tan_email": "TAN-User1@arztkonsultation.de",
        "tan_data_agreed": True,
        "call_type": "tan",
        "additional_callees": [
            {
                "user_id": 59288 #kann weggelassen werden, wenn der Account für den API-Call der ak-User ist, der auch dem Call beitreten soll.
            }
        ]
    }

    call = fetch('POST', '/api/v2/calls', call_data, token)

    call_id = call['data']['id']
    tan = next(user['tan'] for user in call['data']['users'] if user['role_name'] == 'tanuser')

    print('Call')
    print(json.dumps(call['data'], indent=2))

    created_at_timestamp = call['data']['created_at']
    created_at = datetime.utcfromtimestamp(created_at_timestamp).strftime('%Y-%m-%d %H:%M:%S')

    participants = call['data']['users']
    print(f"Der Anruf wurde erstellt am: {created_at} UTC")
    print("Teilnehmer:")
    for user in participants:
        print(f"  - Name: {user['user_name']}, Rolle: {user['role_name']}, E-Mail: {user['email']}")

        if user['role_name'] == 'tanuser':
            tan = user['tan']
            patient_url = f"https://app-sandbox.arztkonsultation.de/video/{call_id}?tan={tan}"
            print(f"  - Link direkt ins Video für {user['user_name']}: {patient_url}")
        else:
            if user['tan']:
                arzt_url = f"https://app-sandbox.arztkonsultation.de/video/{call_id}?tan={user['tan']}"
                print(f"  - Link direkt ins Video für {user['user_name']}: {arzt_url}")
            else:
                arzt_url = f"https://app-sandbox.arztkonsultation.de/video/{call_id}"
                print(f"  - Link direkt ins Video für {user['user_name']}: {arzt_url}")

    return {'callId': call_id, 'tan': tan}

# Aufruf der create_call-Funktion mit dem erhaltenen Token
result = create_call(token)
print(result)


Call
{
  "id": 23013,
  "created_at": 1718104654,
  "updated_at": 1718104654,
  "deleted_at": null,
  "settings": {
    "queue_type": "",
    "offline": [],
    "timings": {
      "lead_time_create": 0,
      "lead_time_cancel": 0
    }
  },
  "parent": null,
  "children": null,
  "begin": null,
  "duration": 15,
  "room": "REyhmO0hOKHkwKhe2cMr0h1QtezQ4s",
  "status": 0,
  "extid": null,
  "description": null,
  "extendable": null,
  "source": null,
  "users": [
    {
      "id": 59373,
      "user_name": "TAN-User1",
      "status": 19,
      "gender": "notset",
      "company_id": null,
      "company_name": null,
      "ousubsidiary_id": null,
      "oulocation_id": null,
      "department_id": null,
      "department_name": null,
      "role_id": 1,
      "role_name": "tanuser",
      "tan": "HEYV-Eitj-EwYj",
      "tan_valid_until": 1726444800,
      "log_id": null,
      "email": "tan-user1@arztkonsultation.de",
      "mobile": null,
      "consentstatus": {
        "tan_descript

## Videocall zwischen *einem* ak-User & *zwei* TAN-Usern (z.B. Arzt/Ärztin zu Patient:in)

Dieser Code zeigt, wie man einen Vidoecall zwischen einem ak-User (z.B. Arzt/Ärztin) und zwei TAN-Usern (z.B. Patient:in und Angehörige Person) erstellt.

Je nach Rolle, kann der Account, der den Call erzeugt hat auch Teilnehmer:in sein (z.B. wenn der Account die Rolle "Arzt" hat)

In [ ]:
import requests
import json
from datetime import datetime


def create_call(token):
    call_data = {
        "duration": 15,
        "tan_username": "TAN-User1",
        "tan_email": "TAN-User1@arztkonsultation.de",
        "tan_data_agreed": True,
        "call_type": "tan",
        "additional_callees": [
            {
                "name": "TAN2",
                "user_id": 59288 #kann weggelassen werden, wenn der Account für den API-Call der ak-User ist, der auch dem Call beitreten soll.

            }
        ]
    }

    call = fetch('POST', '/api/v2/calls', call_data, token)

    call_id = call['data']['id']
    tan = next(user['tan'] for user in call['data']['users'] if user['role_name'] == 'tanuser')

    print('Call')
    print(json.dumps(call['data'], indent=2))

    created_at_timestamp = call['data']['created_at']
    created_at = datetime.utcfromtimestamp(created_at_timestamp).strftime('%Y-%m-%d %H:%M:%S')

    participants = call['data']['users']
    print(f"Der Anruf wurde erstellt am: {created_at} UTC")
    print("Teilnehmer:")
    for user in participants:
        print(f"  - Name: {user['user_name']}, Rolle: {user['role_name']}, E-Mail: {user['email']}")

        if user['role_name'] == 'tanuser':
            tan = user['tan']
            patient_url = f"https://app-sandbox.arztkonsultation.de/video/{call_id}?tan={tan}"
            print(f"  - Link direkt ins Video für {user['user_name']}: {patient_url}")
        else:
            if user['tan']:
                arzt_url = f"https://app-sandbox.arztkonsultation.de/video/{call_id}?tan={user['tan']}"
                print(f"  - Link direkt ins Video für {user['user_name']}: {arzt_url}")
            else:
                arzt_url = f"https://app-sandbox.arztkonsultation.de/video/{call_id}"
                print(f"  - Link direkt ins Video für {user['user_name']}: {arzt_url}")

    return {'callId': call_id, 'tan': tan}

# Aufruf der create_call-Funktion mit dem erhaltenen Token
result = create_call(token)
print(result)


## Videocall zwischen *zwei* ak-User und *einem* TAN-User
In dem nachfolgenden Beispiel, erstellt der Admin-Account einen VideoCall mit einem TAN-User ("TAN-User1") und zwei ak-User.

Die ak-User können dem Videocall entweder über die CallID beitreten oder über eine TAN.

Wenn über die CallId beigetreten wird, dann muss sich der User ggf. nochmal bei ak authentifizieren (E-Mail & Passwort oder SSO).

Der Code gibt die URLs für alle Teilnehmenden aus.

### Zutritt der ak-User via Call-ID

In dem nachfolgenden Beispiel, erstellt der Admin-Account einen VideoCall mit einem TAN-User ("TAN-User1") und zwei ak-User. Die ak-User können dem Videocall via Call-ID beitreten (z.B. https://app-sandbox.arztkonsultation.de/video/23191) und müssen sich dann ggf. vorher authentifizieren.

Der Code gibt die URLs aller Teilnehmeden aus. Der Link für den Admin-User kann ignoriert werden

In [21]:
import json
from datetime import datetime

def create_call(token):
    call_data = {
        "duration": 15,
        "tan_username": "TAN-User1",
        "call_type": "tan",
        "additional_callees": [
            {
                "user_id": 59288 #Ggf. muss nur eine User-Id hinzugefügt werden, wenn der Account für den API-Call der ak-User ist, der auch dem Call beitreten soll.
            },
            {
                "user_id": 59351
            }
        ]
    }

    call = fetch('POST', '/api/v2/calls', call_data, token)

    call_id = call['data']['id']

    print('Call')
    print(json.dumps(call['data'], indent=2))

    created_at_timestamp = call['data']['created_at']
    created_at = datetime.utcfromtimestamp(created_at_timestamp).strftime('%Y-%m-%d %H:%M:%S')

    participants = call['data']['users']
    print(f"Der Anruf wurde erstellt am: {created_at} UTC")
    print("Teilnehmer:")
    for user in participants:
        print(f"  - Name: {user['user_name']}, Rolle: {user['role_name']}, E-Mail: {user['email']}")
        if user['role_name'] == 'tanuser':
            tan = user['tan']
            patient_url = f"https://{hostname}/video/{call['data']['id']}?tan={tan}"
            print(f"  - Link direkt ins Video für {user['user_name']}: {patient_url}")
        elif user['role_name'] == 'callee':
            arzt_url = f"https://{hostname}/video/{call['data']['id']}"
            print(f"  - Link direkt ins Video für {user['user_name']}: {arzt_url}")


# Aufruf der create_call-Funktion mit dem erhaltenen Token
create_call(token)


Call
{
  "id": 23244,
  "created_at": 1718190961,
  "updated_at": 1718190961,
  "deleted_at": null,
  "settings": {
    "queue_type": "",
    "offline": [],
    "timings": {
      "lead_time_create": 0,
      "lead_time_cancel": 0
    }
  },
  "parent": null,
  "children": null,
  "begin": null,
  "duration": 15,
  "room": "0CpkiEtwZ4bqkmG7Vdu90tlHpvgUfF",
  "status": 0,
  "extid": null,
  "description": null,
  "extendable": null,
  "source": null,
  "users": [
    {
      "id": 60014,
      "user_name": "TAN-User1",
      "status": 19,
      "gender": "notset",
      "company_id": null,
      "company_name": null,
      "ousubsidiary_id": null,
      "oulocation_id": null,
      "department_id": null,
      "department_name": null,
      "role_id": 1,
      "role_name": "tanuser",
      "tan": "KA4p-6pLZ-GVvU",
      "tan_valid_until": 1726531200,
      "log_id": null,
      "email": "",
      "mobile": null,
      "consentstatus": {
        "tan_descriptions_agreed": false,
        

### Zutritt der ak-User via TAN

In dem nachfolgenden Beispiel, erstellt der Admin-Account einen VideoCall mit einem TAN-User ("TAN-User1") und zwei ak-User. Die ak-User können dem VideoCall wie TAN beitreten.

Der Code gibt die URL aller Teilnehmeden aus. Der Link für den Admin-User kann ignoriert werden.  

In [5]:
import json
from datetime import datetime

def create_call(token):
    call_data = {
        "duration": 15,
        "tan_username": "TAN-User1",
        "call_type": "tan",
        "additional_callees": [
            {
                "user_id": 59288 #Ggf. muss nur eine User-Id hinzugefügt werden, wenn der Account für den API-Call der ak-User ist, der auch dem Call beitreten soll.
            },
            {
                "user_id": 59351
            }
        ]
    }

    call = fetch('POST', '/api/v2/calls', call_data, token)

    call_id = call['data']['id']

    print('Call')
    print(json.dumps(call['data'], indent=2))

    created_at_timestamp = call['data']['created_at']
    created_at = datetime.utcfromtimestamp(created_at_timestamp).strftime('%Y-%m-%d %H:%M:%S')

    participants = call['data']['users']
    print(f"Der Anruf wurde erstellt am: {created_at} UTC")
    print("Teilnehmer:")
    for user in participants:
        print(f"  - Name: {user['user_name']}, Rolle: {user['role_name']}, E-Mail: {user['email']}")
        tan = user['tan'] if 'tan' in user else None
        if tan:
            url = f"https://{hostname}/video/{call['data']['id']}?tan={tan}"
        else:
            url = f"https://{hostname}/video/{call['data']['id']}"
        print(f"  - Link direkt ins Video für {user['user_name']}: {url}")


# Aufruf der create_call-Funktion mit dem erhaltenen Token
create_call(token)


Call
{
  "id": 23424,
  "created_at": 1718273978,
  "updated_at": 1718273978,
  "deleted_at": null,
  "settings": {
    "queue_type": "",
    "offline": [],
    "timings": {
      "lead_time_create": 0,
      "lead_time_cancel": 0
    }
  },
  "parent": null,
  "children": null,
  "begin": null,
  "duration": null,
  "room": "hlLCpiViTTMIKsYrWv0UfYNoVZ6DSq",
  "status": 0,
  "extid": null,
  "description": null,
  "extendable": null,
  "source": null,
  "users": [
    {
      "id": 60634,
      "user_name": "TAN-User1",
      "status": 19,
      "gender": "notset",
      "company_id": null,
      "company_name": null,
      "ousubsidiary_id": null,
      "oulocation_id": null,
      "department_id": null,
      "department_name": null,
      "role_id": 1,
      "role_name": "tanuser",
      "tan": "ShPs-AeLb-Ejra",
      "tan_valid_until": 1726617600,
      "log_id": null,
      "email": "",
      "mobile": null,
      "consentstatus": {
        "tan_descriptions_agreed": false,
      

## Videocall zwischen *zwei* ak-Usern (z.B. Ärzte-Konsil)
In dem nachfolgenden Beispiel, erstellt der Admin-Account einen VideoCall zwischen zwei ak-Usern.

***Es muss dennoch ein TAN-User hinzugefügt werden, damit die ak-User dem Call zugewiesen werden und dem Call beitreten können.***

Die ak-User können dem Videocall entweder über die CallID beitreten oder über eine TAN. Wenn via CallId beigetreten wird, dann muss sich der User ggf. bei ak authentifizieren.

Der Code gibt die URLs für alle Teilnehmenden aus.

### Zutritt der ak-User via Call-ID

In dem nachfolgenden Beispiel, wird ein Videocall zwischen zwei ak-Usern (z.B. zwei Ärzten) erstellt.

Die ak-User können dem Videocall via Call-ID beitreten (z.B. https://app-sandbox.arztkonsultation.de/video/23191) und müssen sich dann ggf. vorher authentifizieren.

Der Code gibt die URLs aller Teilnehmeden aus. Der Link für den Admin-User kann ignoriert werden

**Info:** ***Es muss immer auch zusätzlich ein TAN User hinzugefügt werden, da sonst die zusätzlichen Users dem Call nicht hinzugefügt werden!***

In [ ]:
import json
from datetime import datetime

def create_call(token):
    call_data = {
        "duration": 15,
        "tan_username": "TAN-User1",
        "call_type": "tan",
        "additional_callees": [
            {
                "user_id": 59288
            },
            {
                "user_id": 59351
            }
        ]
    }

    call = fetch('POST', '/api/v2/calls', call_data, token)

    call_id = call['data']['id']

    print('Call')
    print(json.dumps(call['data'], indent=2))

    created_at_timestamp = call['data']['created_at']
    created_at = datetime.utcfromtimestamp(created_at_timestamp).strftime('%Y-%m-%d %H:%M:%S')

    participants = call['data']['users']
    print(f"Der Anruf wurde erstellt am: {created_at} UTC")
    print("Teilnehmer:")
    for user in participants:
        print(f"  - Name: {user['user_name']}, Rolle: {user['role_name']}, E-Mail: {user['email']}")
        if user['role_name'] == 'tanuser':
            tan = user['tan']
            patient_url = f"https://{hostname}/video/{call['data']['id']}?tan={tan}"
            print(f"  - Link direkt ins Video für {user['user_name']}: {patient_url}")
        elif user['role_name'] == 'callee':
            arzt_url = f"https://{hostname}/video/{call['data']['id']}"
            print(f"  - Link direkt ins Video für {user['user_name']}: {arzt_url}")


# Aufruf der create_call-Funktion mit dem erhaltenen Token
create_call(token)


### Zutritt der ak-User via TAN

In dem nachfolgenden Beispiel, erstellt der Admin-Account einen VideoCall mit zwei ak-Usern. Die ak-User können dem VideoCall über eine TAN beitreten.


Der Code gibt die URL aller Teilnehmeden aus. Der Link für den Admin-User kann ignoriert werden.  
**Text fett markieren**
**Info:** ***Es muss immer auch zusätzlich ein TAN User hinzugefügt werden, da sonst die zusätzlichen Users dem Call nicht hinzugefügt werden!***

In [ ]:
import json
from datetime import datetime

def create_call(token):
    call_data = {
        "duration": 15,
        "tan_username": "TAN-User1",
        "call_type": "tan",
        "additional_callees": [
            {
                "user_id": 59288
            },
            {
                "user_id": 59351
            }
        ]
    }

    call = fetch('POST', '/api/v2/calls', call_data, token)

    call_id = call['data']['id']

    print('Call')
    print(json.dumps(call['data'], indent=2))

    created_at_timestamp = call['data']['created_at']
    created_at = datetime.utcfromtimestamp(created_at_timestamp).strftime('%Y-%m-%d %H:%M:%S')

    participants = call['data']['users']
    print(f"Der Anruf wurde erstellt am: {created_at} UTC")
    print("Teilnehmer:")
    for user in participants:
        print(f"  - Name: {user['user_name']}, Rolle: {user['role_name']}, E-Mail: {user['email']}")
        tan = user['tan'] if 'tan' in user else None
        if tan:
            url = f"https://{hostname}/video/{call['data']['id']}?tan={tan}"
        else:
            url = f"https://{hostname}/video/{call['data']['id']}"
        print(f"  - Link direkt ins Video für {user['user_name']}: {url}")


# Aufruf der create_call-Funktion mit dem erhaltenen Token
create_call(token)


# Abrufen von Benutzern und Anruf-Logs


## Benutzer abrufen

Dieses Beispiel ruft alle Benutzer ab und gibt die User-ID, den Namen und die

Company-ID aus.


In [6]:
import json
import requests
def getAllUsers(token):
    response = fetch('GET', '/api/v2/users?per_page=100', token=token)

    if 'data' in response and isinstance(response['data'], list):
        users = response['data']
    else:
        print("Fehler: Ungültige API-Antwort")
        return {}

    userObj = {}
    for user in users:
        user_id = user.get('id')
        if user_id:
            userObj[user_id] = {
                'display': user.get('name', ''),
                'company': user.get('company_id', '')
            }

    return userObj

users = getAllUsers(token)
formatted_output = json.dumps(users, indent=2)
print(formatted_output)


{
  "58877": {
    "display": "Admin Code Beispiele",
    "company": 827
  },
  "58928": {
    "display": "Testuser2",
    "company": 827
  },
  "59288": {
    "display": "Arzt1 Code-Beispiele",
    "company": 827
  },
  "59351": {
    "display": "Arzt2 Code Beispiel",
    "company": 827
  }
}


## Call Logs abrufen



In [34]:
import requests
from datetime import datetime
import pandas as pd

def getAllUsers(token):
    response = fetch('GET', '/api/v2/users?per_page=100', token=token)

    if 'data' in response and isinstance(response['data'], list):
        users = response['data']
    else:
        print("Fehler: Ungültige API-Antwort")
        return {}

    userObj = {}
    for user in users:
        user_id = user.get('id')
        if user_id:
            userObj[user_id] = {
                'display': user.get('name', ''),
                'company': user.get('company_id', '')
            }

    return userObj

def getCallLogs(token, userStore):
    logs = fetch('GET', '/api/v2/calllogs?per_page=10000', {}, token)['data']
    formatted_logs = []

    for log in logs:
        created_at = datetime.utcfromtimestamp(log['created_at']).strftime('%Y-%m-%d %H:%M:%S')
        updated_at = datetime.utcfromtimestamp(log['updated_at']).strftime('%Y-%m-%d %H:%M:%S')
        deleted_at = datetime.utcfromtimestamp(log['deleted_at']).strftime('%Y-%m-%d %H:%M:%S') if log['deleted_at'] else None
        end = datetime.utcfromtimestamp(log.get('end', 0)).strftime('%Y-%m-%d %H:%M:%S') if log.get('end') else None
        user_id = log['user_id']
        user_info = userStore.get(user_id, {})
        user = user_info.get('display')
        company = user_info.get('company')
        start = datetime.utcfromtimestamp((log['end'] - log['duration']) if log.get('duration') and log.get('end') else 0).strftime('%Y-%m-%d %H:%M:%S')

        formatted_log = {
            'log id': log['id'],
            'call_id': log['call_id'],
            'created_at': created_at,
            'updated_at': updated_at,
            'deleted_at': deleted_at,
            'start': start,
            'end': end,
            'duration in seconds': log['duration'],
            'user_id': user_id,
            'user': user,
            'company id': company
        }
        formatted_logs.append(formatted_log)

    return formatted_logs

# Aufruf der Funktionen
token = login()
userStore = getAllUsers(token)
logs = getCallLogs(token, userStore)

# Daten in ein DataFrame laden
df = pd.DataFrame(logs)

# DataFrame anzeigen
df


,log id,call_id,created_at,updated_at,deleted_at,start,end,duration in seconds,user_id,user,company id
0,19453,22968,2024-06-11 09:48:12,2024-06-11 09:48:26,None,2024-06-11 09:48:12,2024-06-11 09:48:26,14,58877,Admin Code Beispiele,827.0
1,19456,22968,2024-06-11 10:51:27,2024-06-11 10:51:50,None,2024-06-11 10:51:27,2024-06-11 10:51:50,23,58877,Admin Code Beispiele,827.0
2,19461,23008,2024-06-11 11:10:01,2024-06-11 11:10:04,None,2024-06-11 11:10:01,2024-06-11 11:10:04,3,59351,Arzt2 Code Beispiel,827.0
3,19558,23191,2024-06-12 09:05:51,2024-06-12 09:07:12,None,2024-06-12 09:05:51,2024-06-12 09:07:12,81,59288,Arzt1 Code-Beispiele,827.0
4,19559,23191,2024-06-12 09:07:12,2024-06-12 09:08:16,None,2024-06-12 09:07:12,2024-06-12 09:08:16,64,59288,Arzt1 Code-Beispiele,827.0
5,19564,23191,2024-06-12 09:08:31,2024-06-12 09:09:02,None,2024-06-12 09:08:31,2024-06-12 09:09:02,31,59916,None,NaN
6,19568,23191,2024-06-12 09:10:59,2024-06-12 09:11:44,None,2024-06-12 09:10:59,2024-06-12 09:11:44,45,59351,Arzt2 Code Beispiel,827.0
7,19592,23226,2024-06-12 10:48:17,2024-06-12 10:54:30,None,2024-06-12 10:48:17,2024-06-12 10:54:30,373,59996,None,NaN
8,19593,23249,2024-06-12 11:29:15,2024-06-12 11:31:18,None,2024-06-12 11:29:15,2024-06-12 11:31:18,123,60019,None,NaN
9,19598,23249,2024-06-12 11:30:19,2024-06-12 11:31:12,None,2024-06-12 11:30:19,2024-06-12 11:31:12,53,59288,Arzt1 Code-Beispiele,827.0
